In [ ]:
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import torch

DEVICE = torch.device('cuda')

INPUT_WIDTH = 224
INPUT_HEIGHT = 224

paths_labeled = list(Path('train/simpsons_dataset').glob('**/*.jpg'))
paths_test = list(Path('testset/testset').glob('**/*.jpg'))

label_encoder = LabelEncoder().fit([f.parent.name for f in paths_labeled])

from torchvision import transforms

pre_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

from sklearn.model_selection import train_test_split
paths_train, paths_val = train_test_split(paths_labeled)

from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision

class SimpsonsDataset(Dataset):
    def __init__(self, filepaths, has_labels=True):
        super().__init__()
        self.filepaths = sorted(filepaths)
        self.has_labels = has_labels
        self.labels = [label_encoder.transform([f.parent.name])[0] for f in self.filepaths] if has_labels else None

    def __getitem__(self, index):
        # Датасет с отражениями
        img = Image.open(self.filepaths[index // 2])
        if index % 2 == 1:
            img = img.transpose(method=Image.FLIP_LEFT_RIGHT)
        X = pre_transforms(img)
        return (X, self.labels[index // 2]) if self.has_labels else X

    def __len__(self):
        # Датасет с отражениями
        return 2 * len(self.filepaths)

dataset_train = SimpsonsDataset(paths_train)
dataset_val = SimpsonsDataset(paths_val)
dataset_test = SimpsonsDataset(paths_test, has_labels=False)

import matplotlib.pyplot as plt

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
images, labels = iter(DataLoader(dataset_train, batch_size=4)).next()
imshow(torchvision.utils.make_grid(images))
print(label_encoder.inverse_transform(labels))

In [ ]:
from tqdm import tqdm

def train_one_epoch(model, data_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    model.train()
    for x_batch, y_batch in tqdm(data_loader, 'train epoch batches'):
        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * x_batch.size(0)
        running_corrects += torch.sum(preds == y_batch).item()
        processed_size += x_batch.size(0)
    
    return running_loss / processed_size, running_corrects / processed_size

In [ ]:
from tqdm import tqdm

def validate_one_epoch(model, data_loader, criterion):
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    model.eval()
    with torch.no_grad():
        for x_batch, y_batch in tqdm(data_loader, 'validate epoch batches'):
            x_batch = x_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)

            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)

            preds = torch.argmax(outputs, 1)
            running_loss += loss.item() * x_batch.size(0)
            running_corrects += torch.sum(preds == y_batch).item()
            processed_size += x_batch.size(0)
    
    return running_loss / processed_size, running_corrects / processed_size

In [ ]:
from tqdm import tqdm

def run_predict(model, data_loader):
    logits = []
    model.eval()
    with torch.no_grad():
        for x_batch in tqdm(data_loader, 'test epoch batches'):
            x_batch = x_batch.to(DEVICE)
            outputs = model(x_batch)
            preds = torch.argmax(outputs, 1)
            logits.append(preds)
    
    return torch.cat(logits)

In [ ]:
from torch import nn
from torchvision import models

model = models.alexnet(pretrained=True).to(DEVICE)
model.classifier[6] = nn.Linear(4096, len(label_encoder.classes_)).to(DEVICE)
for p in model.features[:-5].parameters():
    p.requires_grad_(False)

history = []
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True)
loader_val = DataLoader(dataset_val, batch_size=256)

for _ in tqdm(range(20), 'epochs'):
    train_loss, train_acc = train_one_epoch(model, loader_train, criterion, optimizer)
    val_loss, val_acc = validate_one_epoch(model, loader_val, criterion)
    history.append((train_loss, train_acc, val_loss, val_acc))

In [ ]:
train_loss, train_acc, val_loss, val_acc = zip(*history)

plt.plot(train_loss, label='train_loss')
plt.plot(val_loss, label='val_loss')
plt.legend(loc='best')
plt.show()

plt.plot(train_acc, label='train_acc')
plt.plot(val_acc, label='val_acc')
plt.legend(loc='best')
plt.show()

In [ ]:
preds = run_predict(model, DataLoader(dataset_test, batch_size=256))

In [ ]:
import pandas as pd
pd.DataFrame({
    'Id': [f.name for f in dataset_test.filepaths],
    'Expected': label_encoder.inverse_transform(preds.cpu().numpy())
}).to_csv('submission.csv', index=None)